In [1]:
from dendron import *
from dendron.actions.generate_action import GenerateAction
from dendron.configs.hflm_action_config import HFLMActionConfig

In [2]:
cfg = HFLMActionConfig(
        model="microsoft/Phi-3-mini-4k-instruct",
        input_key="in",
        output_key="out",
        temperature=0.0,
        device="cpu",
        parallelize=False
    )

node = GenerateAction("lm_action", cfg)
tree = BehaviorTree("causal-lm-tree", node)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
prompt = """{system}
### Instruction:
{query}

### Response:
"""

In [4]:
import re

def trunc(node):
    out_str = node.blackboard[node.output_key]
    m = re.search(r'^Yes\.', out_str)
    if m:
        node.blackboard[node.output_key] = m.group(0).rstrip('.') 

tree.root.add_post_tick(trunc)

In [5]:
tree.blackboard["in"] = prompt.format(system="You are busy and don't have much time to be verbose.", 
                                      query="The following is a headline from a newspaper: 'History-making SpaceX booster mostly destroyed in post-flight topple.' Is this about a spacecraft? Give a one-word yes or no.")

In [6]:
status = tree.tick_once()
if status == NodeStatus.SUCCESS:
     print(tree.blackboard["out"])

Both `max_new_tokens` (=16) and `max_length`(=329) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Yes


In [7]:
print(tree.blackboard)

Key              | Description      | Type             | Value            |
in               | Autogenerated en | <class 'str'>    | You are busy and | 
out              | Autogenerated en | <class 'str'>    | Yes              | 



In [8]:
tree.blackboard['out']

'Yes'